# GIFT-Riemann: Log-Dependent Coefficient Analysis

## Testing Gemini's Hypothesis: a_i(n) = a_i^inf + b_i / log(n)

**Motivation**: Coefficients vary ~50% across fitting ranges, but error decreases with n.
This suggests coefficients are functions of n with logarithmic dependence.

### Experiments

1. **Windowed Coefficient Extraction**: Map how coefficients vary with n
2. **Functional Form Fitting**: Test different models for a_i(n)
3. **Stabilization Test**: Does a_i(n) * log(n) stabilize?
4. **GIFT Asymptotic Limit**: Do a_i^inf match GIFT predictions?
5. **Out-of-Sample Validation**: Does log-corrected model generalize better?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit, minimize
from scipy import stats
from typing import List, Dict, Tuple
import json
import warnings
warnings.filterwarnings('ignore')

print("Log-Dependent Coefficient Analysis")
print("="*50)

## 1. Load Riemann Zeros Data

In [ ]:
# First 200 high-precision Riemann zeros (Odlyzko)
# For full analysis, load from LMFDB or extended tables

RIEMANN_ZEROS_200 = np.array([
    14.134725142, 21.022039639, 25.010857580, 30.424876126, 32.935061588,
    37.586178159, 40.918719012, 43.327073281, 48.005150881, 49.773832478,
    52.970321478, 56.446247697, 59.347044003, 60.831778525, 65.112544048,
    67.079810529, 69.546401711, 72.067157674, 75.704690699, 77.144840069,
    79.337375020, 82.910380854, 84.735492981, 87.425274613, 88.809111208,
    92.491899271, 94.651344041, 95.870634228, 98.831194218, 101.317851006,
    103.725538040, 105.446623052, 107.168611184, 111.029535543, 111.874659177,
    114.320220915, 116.226680321, 118.790782866, 121.370125002, 122.946829294,
    124.256818554, 127.516683880, 129.578704200, 131.087688531, 133.497737203,
    134.756509753, 138.116042055, 139.736208952, 141.123707404, 143.111845808,
    146.000982487, 147.422765343, 150.053520421, 150.925257612, 153.024693811,
    156.112909294, 157.597591818, 158.849988171, 161.188964138, 163.030709687,
    165.537069188, 167.184439978, 169.094515416, 169.911976479, 173.411536520,
    174.754191523, 176.441434298, 178.377407776, 179.916484020, 182.207078484,
    184.874467848, 185.598783678, 187.228922584, 189.416158656, 192.026656361,
    193.079726604, 195.265396680, 196.876481841, 198.015309676, 201.264751944,
    202.493594514, 204.189671803, 205.394697202, 207.906258888, 209.576509717,
    211.690862595, 213.347919360, 214.547044783, 216.169538508, 219.067596349,
    220.714918839, 221.430705555, 224.007000255, 224.983324670, 227.421444280,
    229.337413306, 231.250188700, 231.987235253, 233.693404179, 236.524229666,
    # Continue with more zeros...
    237.769820481, 239.555477887, 241.049157569, 242.823072013, 244.070898497,
    247.136990075, 248.101990394, 249.573689645, 251.014947795, 253.069867484,
    255.306256455, 256.380713694, 258.610439492, 259.874406930, 260.805084505,
    263.573893905, 265.557872729, 266.614973803, 267.991990258, 269.970449024,
    271.494055431, 273.459462056, 275.587492935, 276.452089969, 278.250743530,
    279.229250928, 282.465114665, 283.211185733, 284.835963981, 286.667445561,
    287.911920501, 289.579854929, 291.846298787, 293.558434139, 294.965369619,
    295.573254879, 297.977936943, 299.840328377, 301.649325464, 302.696883037,
    304.864371340, 305.728915917, 307.219491329, 310.109425021, 311.165140617,
    312.427801347, 313.986479905, 315.473679588, 317.734805942, 318.853106408,
    321.163290044, 322.144558814, 323.466969497, 325.226615429, 327.481221349,
    329.032072453, 329.937133793, 331.478698442, 333.645564502, 335.018678555,
    336.841994648, 338.321738098, 339.858075206, 341.042261111, 342.172650177,
    345.223258450, 346.468971658, 347.782292065, 348.617138311, 350.418087699,
    351.878517259, 353.485917256, 356.017390371, 357.151899048, 357.955882560,
    359.743560107, 361.221854729, 363.350273325, 364.136294638, 365.447665523,
    367.477573213, 369.235368157, 370.503915498, 373.060928210, 373.863703042,
    375.825912862, 376.323351498, 378.437031867, 380.319000067, 381.800188279,
    382.454873426, 384.956116708, 385.861284815, 387.222896350, 389.517210868,
    391.127928484, 392.246041641, 393.427082778, 395.582870348, 396.419234500
], dtype=np.float64)

gamma = RIEMANN_ZEROS_200
N = len(gamma)
print(f"Loaded {N} Riemann zeros")
print(f"Range: gamma_1 = {gamma[0]:.4f} to gamma_{N} = {gamma[-1]:.4f}")

## 2. GIFT Constants and Predictions

In [ ]:
# GIFT Topological Constants
GIFT = {
    'dim_G2': 14,
    'b2': 21,
    'b3': 77,
    'H_star': 99,
    'rank_E8': 8,
    'dim_J3O': 27,
    'N_gen': 3,
    'h_G2': 6,  # G2 Coxeter number
    'dim_E7_fund': 56,  # Fundamental rep of E7
    'Weyl': 5,
}

# GIFT predictions for asymptotic coefficients
GIFT_ASYMPTOTIC = {
    5:  GIFT['N_gen'] / GIFT['h_G2'],                  # 3/6 = 0.5
    8:  GIFT['dim_E7_fund'] / GIFT['H_star'],          # 56/99 ≈ 0.566
    13: -GIFT['dim_G2'] / GIFT['H_star'],              # -14/99 ≈ -0.141
    27: 1 / GIFT['dim_J3O'],                           # 1/27 ≈ 0.037
    'c': GIFT['H_star'] / GIFT['Weyl'],                # 99/5 = 19.8
}

print("GIFT Asymptotic Predictions:")
print("="*40)
for key, val in GIFT_ASYMPTOTIC.items():
    if key == 'c':
        print(f"  c^inf = {val:.6f}")
    else:
        print(f"  a_{key}^inf = {val:.6f}")

## 3. Experiment 1: Windowed Coefficient Extraction

Fit the standard recurrence in sliding windows to observe how coefficients evolve with n.

In [ ]:
def fit_recurrence(gamma: np.ndarray, lags: List[int], 
                   start: int = None, end: int = None) -> np.ndarray:
    """
    Fit linear recurrence: gamma_n = sum_i a_i * gamma_{n-lag_i} + c
    Returns coefficient array [a_5, a_8, a_13, a_27, c]
    """
    max_lag = max(lags)
    if start is None:
        start = max_lag
    if end is None:
        end = len(gamma)
    
    X = []
    y = []
    for n in range(start, end):
        row = [gamma[n - lag] for lag in lags] + [1.0]
        X.append(row)
        y.append(gamma[n])
    
    X = np.array(X)
    y = np.array(y)
    
    coeffs, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
    return coeffs


def windowed_analysis(gamma: np.ndarray, lags: List[int] = [5, 8, 13, 27],
                      window_size: int = 50, step: int = 10) -> List[Dict]:
    """
    Fit recurrence in sliding windows.
    Returns list of dicts with coefficients and window center.
    """
    results = []
    max_lag = max(lags)
    
    for start in range(max_lag, len(gamma) - window_size, step):
        end = start + window_size
        n_center = (start + end) // 2
        
        coeffs = fit_recurrence(gamma, lags, start, end)
        
        results.append({
            'n_center': n_center,
            'log_n': np.log(n_center),
            'a5': coeffs[0],
            'a8': coeffs[1],
            'a13': coeffs[2],
            'a27': coeffs[3],
            'c': coeffs[4]
        })
    
    return results


# Run windowed analysis
lags = [5, 8, 13, 27]
windowed = windowed_analysis(gamma, lags, window_size=50, step=5)

print(f"Analyzed {len(windowed)} windows")
print(f"n range: {windowed[0]['n_center']} to {windowed[-1]['n_center']}")

In [ ]:
# Extract arrays for plotting
n_centers = np.array([w['n_center'] for w in windowed])
log_n = np.array([w['log_n'] for w in windowed])
a5_vals = np.array([w['a5'] for w in windowed])
a8_vals = np.array([w['a8'] for w in windowed])
a13_vals = np.array([w['a13'] for w in windowed])
a27_vals = np.array([w['a27'] for w in windowed])
c_vals = np.array([w['c'] for w in windowed])

# Plot coefficient evolution
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

coef_data = [
    (a5_vals, 'a_5', GIFT_ASYMPTOTIC[5], 'tab:blue'),
    (a8_vals, 'a_8', GIFT_ASYMPTOTIC[8], 'tab:orange'),
    (a13_vals, 'a_{13}', GIFT_ASYMPTOTIC[13], 'tab:green'),
    (a27_vals, 'a_{27}', GIFT_ASYMPTOTIC[27], 'tab:red'),
    (c_vals, 'c', GIFT_ASYMPTOTIC['c'], 'tab:purple'),
]

for ax, (vals, name, gift_val, color) in zip(axes.flat, coef_data):
    ax.scatter(log_n, vals, alpha=0.6, s=20, c=color)
    ax.axhline(gift_val, color='red', linestyle='--', label=f'GIFT = {gift_val:.4f}')
    ax.set_xlabel('log(n)')
    ax.set_ylabel(f'${name}$')
    ax.set_title(f'Coefficient ${name}$ vs log(n)')
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)

# Hide last subplot
axes.flat[-1].axis('off')

plt.tight_layout()
plt.savefig('coefficient_evolution.png', dpi=150)
plt.show()

print("Coefficient Evolution Plot saved")

## 4. Experiment 2: Fit Functional Forms

Test different models for coefficient dependence on n:
- **Form A**: a_i(n) = a_i^0 / log(n)
- **Form B**: a_i(n) = a_i^inf + b_i / log(n)
- **Power law**: a_i(n) = a_i^0 * n^alpha

In [ ]:
def fit_form_A(log_n: np.ndarray, a_vals: np.ndarray) -> Tuple[float, float]:
    """Fit a(n) = a0 / log(n). Returns (a0, R^2)."""
    # Linear regression: a * log(n) = a0
    a_times_log = a_vals * log_n
    a0 = np.mean(a_times_log)
    
    # Predictions and R^2
    a_pred = a0 / log_n
    ss_res = np.sum((a_vals - a_pred)**2)
    ss_tot = np.sum((a_vals - np.mean(a_vals))**2)
    r2 = 1 - ss_res / ss_tot if ss_tot > 0 else 0
    
    return a0, r2


def fit_form_B(log_n: np.ndarray, a_vals: np.ndarray) -> Tuple[float, float, float]:
    """Fit a(n) = a_inf + b / log(n). Returns (a_inf, b, R^2)."""
    # Linear regression: a = a_inf + b / log(n)
    X = np.column_stack([np.ones_like(log_n), 1.0 / log_n])
    coeffs, _, _, _ = np.linalg.lstsq(X, a_vals, rcond=None)
    a_inf, b = coeffs
    
    # R^2
    a_pred = a_inf + b / log_n
    ss_res = np.sum((a_vals - a_pred)**2)
    ss_tot = np.sum((a_vals - np.mean(a_vals))**2)
    r2 = 1 - ss_res / ss_tot if ss_tot > 0 else 0
    
    return a_inf, b, r2


def fit_power_law(n_centers: np.ndarray, a_vals: np.ndarray) -> Tuple[float, float, float]:
    """Fit a(n) = a0 * n^alpha. Returns (a0, alpha, R^2)."""
    # Log transform: log(a) = log(a0) + alpha * log(n)
    # Only for positive a values
    mask = a_vals > 0
    if np.sum(mask) < 3:
        return np.nan, np.nan, 0
    
    log_a = np.log(np.abs(a_vals[mask]))
    log_n = np.log(n_centers[mask])
    
    X = np.column_stack([np.ones_like(log_n), log_n])
    coeffs, _, _, _ = np.linalg.lstsq(X, log_a, rcond=None)
    log_a0, alpha = coeffs
    a0 = np.exp(log_a0) * np.sign(np.mean(a_vals))
    
    # R^2 on original scale
    a_pred = a0 * n_centers**alpha
    ss_res = np.sum((a_vals - a_pred)**2)
    ss_tot = np.sum((a_vals - np.mean(a_vals))**2)
    r2 = 1 - ss_res / ss_tot if ss_tot > 0 else 0
    
    return a0, alpha, r2


# Fit all forms for each coefficient
print("Functional Form Comparison")
print("="*80)

results_forms = {}

for name, vals, gift_val in [('a5', a5_vals, GIFT_ASYMPTOTIC[5]),
                              ('a8', a8_vals, GIFT_ASYMPTOTIC[8]),
                              ('a13', a13_vals, GIFT_ASYMPTOTIC[13]),
                              ('a27', a27_vals, GIFT_ASYMPTOTIC[27]),
                              ('c', c_vals, GIFT_ASYMPTOTIC['c'])]:
    
    print(f"\n{name}:")
    print(f"  GIFT prediction: {gift_val:.6f}")
    print(f"  Mean observed:   {np.mean(vals):.6f}")
    print(f"  Std observed:    {np.std(vals):.6f}")
    print(f"  CV (raw):        {np.std(vals)/np.abs(np.mean(vals))*100:.1f}%")
    
    # Form A
    a0_A, r2_A = fit_form_A(log_n, vals)
    print(f"  Form A: a0 = {a0_A:.6f}, R^2 = {r2_A:.4f}")
    
    # Form B
    a_inf_B, b_B, r2_B = fit_form_B(log_n, vals)
    print(f"  Form B: a_inf = {a_inf_B:.6f}, b = {b_B:.6f}, R^2 = {r2_B:.4f}")
    print(f"          GIFT deviation: {abs(a_inf_B - gift_val)/abs(gift_val)*100:.1f}%")
    
    # Power law
    a0_P, alpha_P, r2_P = fit_power_law(n_centers, vals)
    if not np.isnan(a0_P):
        print(f"  Power:  a0 = {a0_P:.6f}, alpha = {alpha_P:.6f}, R^2 = {r2_P:.4f}")
    
    results_forms[name] = {
        'form_A': {'a0': a0_A, 'r2': r2_A},
        'form_B': {'a_inf': a_inf_B, 'b': b_B, 'r2': r2_B},
        'gift_deviation_pct': abs(a_inf_B - gift_val)/abs(gift_val)*100 if gift_val != 0 else None
    }

## 5. Experiment 3: Stabilization Test

Test if normalizing by log(n) reduces coefficient variance.

In [ ]:
def test_stabilization(vals: np.ndarray, log_n: np.ndarray, 
                       a_inf: float = None) -> Dict:
    """
    Test coefficient stabilization.
    
    Compares:
    - Raw CV
    - CV after subtracting asymptotic (Form B)
    - CV of a(n) * log(n) (Form A normalized)
    """
    cv_raw = np.std(vals) / np.abs(np.mean(vals)) * 100 if np.mean(vals) != 0 else np.inf
    
    # Form A normalized: a(n) * log(n)
    vals_A_norm = vals * log_n
    cv_A = np.std(vals_A_norm) / np.abs(np.mean(vals_A_norm)) * 100 if np.mean(vals_A_norm) != 0 else np.inf
    
    # Form B normalized: a(n) - a_inf
    if a_inf is not None:
        residuals_B = vals - a_inf
        # Check if residuals ~ 1/log(n)
        residuals_scaled = residuals_B * log_n
        cv_B = np.std(residuals_scaled) / np.abs(np.mean(residuals_scaled)) * 100 if np.mean(residuals_scaled) != 0 else np.inf
    else:
        cv_B = np.inf
    
    return {
        'cv_raw': cv_raw,
        'cv_form_A_normalized': cv_A,
        'cv_form_B_residual': cv_B,
        'stabilization_A': cv_raw / cv_A if cv_A > 0 else 0,
        'stabilization_B': cv_raw / cv_B if cv_B > 0 else 0,
    }


print("\nStabilization Test")
print("="*60)
print(f"{'Coef':<6} {'CV Raw':>10} {'CV(A)':>10} {'CV(B)':>10} {'Stab(A)':>10} {'Stab(B)':>10}")
print("-"*60)

stabilization_results = {}

for name, vals in [('a5', a5_vals), ('a8', a8_vals), ('a13', a13_vals), 
                   ('a27', a27_vals), ('c', c_vals)]:
    # Get a_inf from Form B fit
    a_inf = results_forms[name]['form_B']['a_inf']
    
    result = test_stabilization(vals, log_n, a_inf)
    stabilization_results[name] = result
    
    print(f"{name:<6} {result['cv_raw']:>9.1f}% {result['cv_form_A_normalized']:>9.1f}% "
          f"{result['cv_form_B_residual']:>9.1f}% {result['stabilization_A']:>9.2f}x "
          f"{result['stabilization_B']:>9.2f}x")

print("\nInterpretation:")
print("  Stabilization > 1 means normalization REDUCES variance")
print("  Stabilization < 1 means normalization INCREASES variance")

## 6. Experiment 4: Full Form B Fit on Original Data

Fit the log-corrected recurrence directly to gamma values.

In [ ]:
def fit_full_form_B(gamma: np.ndarray, lags: List[int] = [5, 8, 13, 27],
                    start: int = None, end: int = None) -> Dict:
    """
    Fit the full Form B model:
    gamma_n = sum_i (a_i^inf + b_i/log(n)) * gamma_{n-lag_i} + c^inf + d/log(n)
    
    Parameters: a_i^inf, b_i for each lag, plus c^inf, d
    Total: 2 * len(lags) + 2 = 10 parameters for 4 lags
    """
    max_lag = max(lags)
    if start is None:
        start = max_lag
    if end is None:
        end = len(gamma)
    
    # Build design matrix
    X = []
    y = []
    n_array = []
    
    for n in range(start, end):
        log_n = np.log(n)
        row = []
        for lag in lags:
            g_lag = gamma[n - lag]
            row.append(g_lag)          # a_i^inf term
            row.append(g_lag / log_n)  # b_i term
        row.append(1.0)           # c^inf
        row.append(1.0 / log_n)   # d
        
        X.append(row)
        y.append(gamma[n])
        n_array.append(n)
    
    X = np.array(X)
    y = np.array(y)
    n_array = np.array(n_array)
    
    # Solve
    coeffs, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
    
    # Parse coefficients
    result = {'model': 'Form B', 'lags': lags}
    for i, lag in enumerate(lags):
        result[f'a{lag}_inf'] = coeffs[2*i]
        result[f'b{lag}'] = coeffs[2*i + 1]
    result['c_inf'] = coeffs[-2]
    result['d'] = coeffs[-1]
    
    # Predictions and errors
    y_pred = X @ coeffs
    errors_pct = np.abs(y_pred - y) / y * 100
    result['mean_error_pct'] = np.mean(errors_pct)
    result['max_error_pct'] = np.max(errors_pct)
    result['std_error_pct'] = np.std(errors_pct)
    result['n_points'] = len(y)
    result['predictions'] = y_pred
    result['actual'] = y
    result['n_array'] = n_array
    
    return result


# Fit standard (constant coefficient) model for comparison
def fit_constant(gamma: np.ndarray, lags: List[int] = [5, 8, 13, 27],
                 start: int = None, end: int = None) -> Dict:
    """Fit constant-coefficient model for comparison."""
    max_lag = max(lags)
    if start is None:
        start = max_lag
    if end is None:
        end = len(gamma)
    
    coeffs = fit_recurrence(gamma, lags, start, end)
    
    # Compute predictions
    y_pred = []
    y_true = []
    for n in range(start, end):
        pred = sum(coeffs[i] * gamma[n - lag] for i, lag in enumerate(lags))
        pred += coeffs[-1]
        y_pred.append(pred)
        y_true.append(gamma[n])
    
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    errors_pct = np.abs(y_pred - y_true) / y_true * 100
    
    result = {
        'model': 'Constant',
        'coefficients': coeffs.tolist(),
        'mean_error_pct': np.mean(errors_pct),
        'max_error_pct': np.max(errors_pct),
        'n_points': len(y_true)
    }
    
    for i, lag in enumerate(lags):
        result[f'a{lag}'] = coeffs[i]
    result['c'] = coeffs[-1]
    
    return result


# Compare models
print("\nModel Comparison: Constant vs Form B")
print("="*60)

lags = [5, 8, 13, 27]

result_const = fit_constant(gamma, lags)
result_formB = fit_full_form_B(gamma, lags)

print(f"\nConstant Coefficient Model:")
print(f"  Mean error: {result_const['mean_error_pct']:.4f}%")
print(f"  Max error:  {result_const['max_error_pct']:.4f}%")
for lag in lags:
    print(f"  a_{lag} = {result_const[f'a{lag}']:.6f} (GIFT: {GIFT_ASYMPTOTIC[lag]:.6f})")
print(f"  c = {result_const['c']:.6f} (GIFT: {GIFT_ASYMPTOTIC['c']:.6f})")

print(f"\nForm B (Log-Corrected) Model:")
print(f"  Mean error: {result_formB['mean_error_pct']:.4f}%")
print(f"  Max error:  {result_formB['max_error_pct']:.4f}%")
for lag in lags:
    a_inf = result_formB[f'a{lag}_inf']
    b = result_formB[f'b{lag}']
    gift = GIFT_ASYMPTOTIC[lag]
    dev = abs(a_inf - gift)/abs(gift)*100 if gift != 0 else 0
    print(f"  a_{lag}(n) = {a_inf:.4f} + {b:.4f}/log(n)  [GIFT: {gift:.4f}, dev: {dev:.1f}%]")
c_inf = result_formB['c_inf']
d = result_formB['d']
print(f"  c(n) = {c_inf:.4f} + {d:.4f}/log(n)  [GIFT: {GIFT_ASYMPTOTIC['c']:.4f}]")

print(f"\nImprovement: {(result_const['mean_error_pct'] - result_formB['mean_error_pct'])/result_const['mean_error_pct']*100:.1f}%")

## 7. Experiment 5: GIFT Asymptotic Limit Test

Fix asymptotic coefficients to GIFT values and only fit the correction terms.

In [ ]:
def fit_form_B_fixed_asymptotic(gamma: np.ndarray, lags: List[int],
                                 gift_asymptotic: Dict,
                                 start: int = None, end: int = None) -> Dict:
    """
    Fit Form B with GIFT asymptotic values FIXED.
    Only b_i (correction terms) are optimized.
    """
    max_lag = max(lags)
    if start is None:
        start = max_lag
    if end is None:
        end = len(gamma)
    
    # Build modified design matrix
    X = []  # For b_i and d only
    y_mod = []  # gamma_n - fixed asymptotic contribution
    
    for n in range(start, end):
        log_n = np.log(n)
        
        # Subtract fixed asymptotic contribution
        y_target = gamma[n]
        for lag in lags:
            y_target -= gift_asymptotic[lag] * gamma[n - lag]
        y_target -= gift_asymptotic['c']  # c^inf
        
        # Build row for correction terms
        row = []
        for lag in lags:
            row.append(gamma[n - lag] / log_n)  # b_i term
        row.append(1.0 / log_n)  # d term
        
        X.append(row)
        y_mod.append(y_target)
    
    X = np.array(X)
    y_mod = np.array(y_mod)
    
    # Solve for correction terms
    correction_coeffs, _, _, _ = np.linalg.lstsq(X, y_mod, rcond=None)
    
    # Compute predictions with full model
    y_pred = []
    y_true = []
    for n in range(start, end):
        log_n = np.log(n)
        pred = gift_asymptotic['c'] + correction_coeffs[-1] / log_n
        for i, lag in enumerate(lags):
            pred += (gift_asymptotic[lag] + correction_coeffs[i] / log_n) * gamma[n - lag]
        y_pred.append(pred)
        y_true.append(gamma[n])
    
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    errors_pct = np.abs(y_pred - y_true) / y_true * 100
    
    result = {
        'model': 'Form B (GIFT fixed)',
        'mean_error_pct': np.mean(errors_pct),
        'max_error_pct': np.max(errors_pct),
        'corrections': {}
    }
    
    for i, lag in enumerate(lags):
        result['corrections'][f'b{lag}'] = correction_coeffs[i]
    result['corrections']['d'] = correction_coeffs[-1]
    
    return result


# Test GIFT-fixed model
print("\nGIFT Asymptotic Limit Test")
print("="*60)
print("Fixing a_i^inf to GIFT predictions, fitting only b_i corrections...")

result_gift_fixed = fit_form_B_fixed_asymptotic(gamma, lags, GIFT_ASYMPTOTIC)

print(f"\nResults with GIFT fixed:")
print(f"  Mean error: {result_gift_fixed['mean_error_pct']:.4f}%")
print(f"  Max error:  {result_gift_fixed['max_error_pct']:.4f}%")
print(f"\n  Fitted corrections (b_i):")
for key, val in result_gift_fixed['corrections'].items():
    print(f"    {key} = {val:.6f}")

print(f"\nComparison:")
print(f"  Constant model error:   {result_const['mean_error_pct']:.4f}%")
print(f"  Form B (free) error:    {result_formB['mean_error_pct']:.4f}%")
print(f"  Form B (GIFT) error:    {result_gift_fixed['mean_error_pct']:.4f}%")

if result_gift_fixed['mean_error_pct'] < result_const['mean_error_pct'] * 2:
    print("\n  -> GIFT asymptotic values are VIABLE")
else:
    print("\n  -> GIFT asymptotic values need adjustment")

## 8. Summary and Conclusions

In [ ]:
print("\n" + "="*70)
print("SUMMARY: LOG-DEPENDENT COEFFICIENT ANALYSIS")
print("="*70)

print("\n1. COEFFICIENT EVOLUTION WITH n:")
print("   Coefficients show systematic variation with n.")
print(f"   CV ranges from {min(stabilization_results[k]['cv_raw'] for k in stabilization_results):.1f}% "
      f"to {max(stabilization_results[k]['cv_raw'] for k in stabilization_results):.1f}%")

print("\n2. BEST FUNCTIONAL FORM:")
# Check which form has best R^2 on average
r2_A = np.mean([results_forms[k]['form_A']['r2'] for k in results_forms])
r2_B = np.mean([results_forms[k]['form_B']['r2'] for k in results_forms])
print(f"   Form A (a/log(n)): avg R^2 = {r2_A:.4f}")
print(f"   Form B (a_inf + b/log(n)): avg R^2 = {r2_B:.4f}")
if r2_B > r2_A:
    print("   -> Form B is preferred (asymptotic + correction)")
else:
    print("   -> Form A is preferred (pure scaling)")

print("\n3. STABILIZATION:")
avg_stab_A = np.mean([stabilization_results[k]['stabilization_A'] for k in stabilization_results])
avg_stab_B = np.mean([stabilization_results[k]['stabilization_B'] for k in stabilization_results])
print(f"   Form A normalization: {avg_stab_A:.2f}x variance reduction")
print(f"   Form B normalization: {avg_stab_B:.2f}x variance reduction")

print("\n4. GIFT ASYMPTOTIC LIMITS:")
gift_deviations = [results_forms[k].get('gift_deviation_pct', 100) for k in results_forms if k != 'c']
avg_gift_dev = np.mean([d for d in gift_deviations if d is not None])
print(f"   Average deviation from GIFT: {avg_gift_dev:.1f}%")
if avg_gift_dev < 20:
    print("   -> GIFT predictions are reasonably close to fitted asymptotic values")
elif avg_gift_dev < 50:
    print("   -> GIFT predictions are in the right ballpark")
else:
    print("   -> GIFT predictions differ significantly from data")

print("\n5. MODEL COMPARISON:")
print(f"   Constant coefficient: {result_const['mean_error_pct']:.4f}% error")
print(f"   Form B (free params): {result_formB['mean_error_pct']:.4f}% error")
print(f"   Form B (GIFT fixed):  {result_gift_fixed['mean_error_pct']:.4f}% error")

print("\n" + "="*70)
print("CONCLUSIONS")
print("="*70)

conclusions = []

if result_formB['mean_error_pct'] < result_const['mean_error_pct'] * 0.9:
    conclusions.append("Log-correction IMPROVES prediction accuracy")
else:
    conclusions.append("Log-correction does NOT significantly improve accuracy")

if r2_B > 0.5:
    conclusions.append("Form B (a_inf + b/log(n)) explains coefficient variation well")
    
if avg_gift_dev < 30:
    conclusions.append("GIFT constants are viable as asymptotic limits")
    
if result_gift_fixed['mean_error_pct'] < result_const['mean_error_pct'] * 1.5:
    conclusions.append("GIFT-fixed model is competitive")

for i, c in enumerate(conclusions, 1):
    print(f"   {i}. {c}")

print("\n" + "="*70)
print("NEXT STEPS")
print("="*70)
print("   1. Test on larger dataset (100k+ zeros)")
print("   2. Implement cross-validation for model selection")
print("   3. Explore higher-order corrections (1/log^2(n))")
print("   4. Test on other L-functions for universality")

In [ ]:
# Export results to JSON
def to_serializable(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, (np.integer, np.floating)):
        return float(obj)
    elif isinstance(obj, dict):
        return {k: to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [to_serializable(v) for v in obj]
    return obj

export_data = {
    'n_zeros': int(N),
    'lags': lags,
    'gift_asymptotic': GIFT_ASYMPTOTIC,
    'constant_model': to_serializable(result_const),
    'form_B_free': {
        'mean_error_pct': float(result_formB['mean_error_pct']),
        'coefficients': {k: float(v) for k, v in result_formB.items() 
                        if k.startswith('a') or k.startswith('b') or k in ['c_inf', 'd']}
    },
    'form_B_gift_fixed': to_serializable(result_gift_fixed),
    'stabilization': to_serializable(stabilization_results),
    'functional_forms': to_serializable(results_forms)
}

with open('log_correction_results.json', 'w') as f:
    json.dump(export_data, f, indent=2)

print("\nResults exported to log_correction_results.json")